In [1]:
# example of a super learner model for binary classification
from numpy import hstack
from numpy import vstack
from numpy import asarray
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [2]:
#create a list of base models
def get_models():
  models = []
  models.append(SVC(kernel='rbf',probability=True,random_state=0))
  models.append(XGBClassifier(max_depth=9, eta = 0.4, gamma = 3, n_rounds = 100))
  return models

In [3]:
# collect out of fold predictions form k-fold cross validation
def get_out_of_fold_predictions(X, y, models):
	meta_X, meta_y = list(), list()
	# define split of data
	kfold = KFold(n_splits=10, shuffle=True)
	# enumerate splits
	for train_ix, test_ix in kfold.split(X):
		fold_yhats = list()
		# get data
		train_X, test_X = X[train_ix], X[test_ix]
		train_y, test_y = y[train_ix], y[test_ix]
		meta_y.extend(test_y)
		# fit and make predictions with each sub-model
		for model in models:
			model.fit(train_X, train_y)
			yhat = model.predict_proba(test_X)
			# store columns
			fold_yhats.append(yhat)
		# store fold yhats as columns
		meta_X.append(hstack(fold_yhats))
	return vstack(meta_X), asarray(meta_y)

In [4]:
# fit all base models on the training dataset
def fit_base_models(X, y, models):
	for model in models:
		model.fit(X, y)

In [5]:
# fit a meta model
def fit_meta_model(X, y):
	model = LogisticRegression(solver='liblinear')
	model.fit(X, y)
	return model

In [6]:
# make predictions with stacked model
def super_learner_predictions(X, models, meta_model):
	meta_X = list()
	for model in models:
		yhat = model.predict_proba(X)
		meta_X.append(yhat)
	meta_X = hstack(meta_X)
	# predict
	return meta_model.predict_proba(meta_X)

In [7]:
import io
import pandas as pd
df = pd.read_csv('X_trainData_column_modified_AMK.csv') #Enter the file name here.
# Dataset is now stored in a Pandas Dataframe

In [8]:
df

DEL_CF_410280_d918T_307_iniB  DEL_CF_4408101_d102C_34_gid  \
0                                0                            0   
1                                0                            0   
2                                0                            0   
3                                0                            0   
4                                0                            0   
...                            ...                          ...   
1355                             0                            0   
1356                             0                            0   
1357                             0                            0   
1358                             0                            0   
1359                             0                            0   

      DEL_F_4408101_d101C_gid_G34G  INS_CF_4242820_i2957G_986_embC  \
0                                0                               0   
1                                0                               0   
2                                0                               0   
3                                0                               0   
4                                0                               0   
...                            ...                             ...   
1355                             0                               0   
1356                             0                               0   
1357                             0                               0   
1358                             0                               0   
1359                             0                               0   

      INS_F_409772_i410ATCT_iniB_G137G  INS_F_4247020_i506CC_embB_G169G  \
0                                    0                                0   
1                                    0                                0   
2                                    0                                0   
3                                    0                                0   
4                                    0                                0   
...                                ...                              ...   
1355                                 0                                0   
1356                                 0                                0   
1357                                 0                                0   
1358                                 0                                0   
1359                                 0                                0   

      INS_F_4247970_i1456GT_embB_G486G  \
0                                    1   
1                                    0   
2                                    0   
3                                    0   
4                                    0   
...                                ...   
1355                                 0   
1356                                 0   
1357                                 0   
1358                                 0   
1359                                 0   

      INS_N_4243642_i409GTCCCGGGGCGCCAC_embA_S137S  INS_P_3074519_G.117_thyA  \
0                                                0                         0   
1                                                0                         1   
2                                                0                         1   
3                                                1                         1   
4                                                0                         1   
...                                            ...                       ...   
1355                                             0                         0   
1356                                             0                         0   
1357                                             0                         0   
1358                                             0                         0   
1359                                             0                        

In [9]:
X = df.iloc[:,:-1].values
X.shape

(1360, 219)

In [10]:
y = df.iloc[:,-1].values
y.shape

(1360,)

In [11]:
# get models
models = get_models()
# get out of fold predictions
meta_X, meta_y = get_out_of_fold_predictions(X, y, models)
print('Meta ', meta_X.shape, meta_y.shape)

[15:48:48] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:48:50] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:48:52] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:48:54] WARNI

In [12]:
# fit base models
fit_base_models(X, y, models)

[15:49:07] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [13]:
# fit the meta model
meta_model = fit_meta_model(meta_X, meta_y)

In [14]:
X_test = pd.read_csv('final_X_testData_column_modified_AMK.csv') #Enter the file name here.
# Dataset is now stored in a Pandas Dataframe

In [15]:
import numpy as np

In [16]:
X_test

DEL_CF_410280_d918T_307_iniB  DEL_CF_4408101_d102C_34_gid  \
0                               0                            0   
1                               0                            0   
2                               0                            0   
3                               0                            0   
4                               0                            0   
..                            ...                          ...   
336                             0                            0   
337                             0                            0   
338                             0                            0   
339                             0                            0   
340                             0                            0   

     DEL_F_4408101_d101C_gid_G34G  INS_CF_4242820_i2957G_986_embC  \
0                               0                               0   
1                               0                               0   
2                               0                               0   
3                               0                               0   
4                               0                               0   
..                            ...                             ...   
336                             0                               0   
337                             0                               0   
338                             0                               0   
339                             0                               0   
340                             0                               0   

     INS_F_409772_i410ATCT_iniB_G137G  INS_F_4247020_i506CC_embB_G169G  \
0                                   0                                0   
1                                   0                                0   
2                                   0                                0   
3                                   0                                0   
4                                   0                                0   
..                                ...                              ...   
336                                 0                                0   
337                                 0                                0   
338                                 0                                0   
339                                 0                                0   
340                                 0                                0   

     INS_F_4247970_i1456GT_embB_G486G  \
0                                   0   
1                                   0   
2                                   0   
3                                   0   
4                                   0   
..                                ...   
336                                 0   
337                                 0   
338                                 0   
339                                 0   
340                                 0   

     INS_N_4243642_i409GTCCCGGGGCGCCAC_embA_S137S  INS_P_3074519_G.117_thyA  \
0                                               0                         0   
1                                               0                         0   
2                                               0                         0   
3                                               0                         0   
4                                               0                         0   
..                                            ...                       ...   
336                                             0                         0   
337                                             0                         0   
338                                             0                         0   
339                                             0                         0   
340                                             0                         0   

     INS_P_3074521_G.115_thyA  ...  ahpC_NC_snp  ahp

In [17]:
X_val = X_test.iloc[:,:-1].values
X_val.shape

(341, 219)

In [18]:
yhat = super_learner_predictions(X_val, models, meta_model)


In [19]:
yhat

array([[0.02845468, 0.97154532],
       [0.03465143, 0.96534857],
       [0.02748864, 0.97251136],
       [0.02798627, 0.97201373],
       [0.02770085, 0.97229915],
       [0.02748864, 0.97251136],
       [0.02873267, 0.97126733],
       [0.03006901, 0.96993099],
       [0.02759802, 0.97240198],
       [0.02880529, 0.97119471],
       [0.02798054, 0.97201946],
       [0.02876635, 0.97123365],
       [0.02791243, 0.97208757],
       [0.0286871 , 0.9713129 ],
       [0.02893111, 0.97106889],
       [0.02876474, 0.97123526],
       [0.02759108, 0.97240892],
       [0.0279582 , 0.9720418 ],
       [0.02779347, 0.97220653],
       [0.02742669, 0.97257331],
       [0.02753295, 0.97246705],
       [0.02798627, 0.97201373],
       [0.02809398, 0.97190602],
       [0.02696873, 0.97303127],
       [0.02739969, 0.97260031],
       [0.0270993 , 0.9729007 ],
       [0.02804349, 0.97195651],
       [0.02705112, 0.97294888],
       [0.0276167 , 0.9723833 ],
       [0.02987958, 0.97012042],
       [0.

In [20]:
submission_data = pd.read_csv('Y_testData_1_nolabels_AMK.csv')

In [21]:
ids=submission_data.iloc[:,:1].values

In [22]:
result=list()

for i in range(len(ids)):
    result.append([ ids[i,0], yhat[i,1]])

result.insert(0, ['ID','AMK'])
result

[['ID', 'AMK'],
 [1, 0.9715453231806765],
 [2, 0.9653485667339359],
 [3, 0.9725113597371002],
 [4, 0.9720137250451613],
 [5, 0.9722991481059459],
 [6, 0.9725113597371002],
 [7, 0.9712673299110756],
 [8, 0.969930987035729],
 [9, 0.972401978515499],
 [10, 0.9711947147946823],
 [11, 0.9720194580859401],
 [12, 0.9712336480410774],
 [13, 0.9720875733875831],
 [14, 0.9713129018013599],
 [15, 0.9710688904423644],
 [16, 0.9712352621828756],
 [17, 0.972408920991997],
 [18, 0.972041803543828],
 [19, 0.9722065281379635],
 [20, 0.9725733065082728],
 [21, 0.9724670510434535],
 [22, 0.9720137250451613],
 [23, 0.9719060233728719],
 [24, 0.9730312740635582],
 [25, 0.9726003098417697],
 [26, 0.9729007030938933],
 [27, 0.9719565091941632],
 [28, 0.9729488793015927],
 [29, 0.9723833006472121],
 [30, 0.9701204207821372],
 [31, 0.9714782910007865],
 [32, 0.9698046353378575],
 [33, 0.971256313008332],
 [34, 0.970541404652967],
 [35, 0.08784395438159748],
 [36, 0.9733381301981646],
 [37, 0.9712673299110756],

In [23]:
output = pd.DataFrame(result)
output.to_csv('AMK_results.csv',header = False, index = False)